# APIM ❤️ OpenAI

## Advanced Load Balancing lab
![flow](../../images/advanced-load-balancing.gif)

Playground to try the advanced load balancing (based on a custom [APIM policy](https://learn.microsoft.com/en-us/azure/api-management/api-management-howto-policies)) to either a list of Azure OpenAI endpoints or mock servers.

This load balancer is based on a custom policy configuration forked from this great [smart load balancer repo](https://github.com/andredewes/apim-aoai-smart-loadbalancing) and adds the following enhancements:
- Loads the load balancer configuration from a named value
- Uses backends to enable the combination with the built-in circuit breaking feature
- The policy doesn't have to be changed to add/modify endpoints or configure the load balancer 
- Dynamicly supports any number of OpenAI endpoints

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [🧪 Test the API using a direct HTTP call](#requests)
- [🧪 Test the API using the Azure OpenAI Python SDK](#sdk)
- [🗑️ Clean up resources](#clean)

### Backlog
- Weighted backend pools
- Stickiness for the stateful [assistants API](https://cookbook.openai.com/examples/assistants_api_overview_python)

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- The ```mock_webapps``` variable sets the list of deployed Web Apps for the mocking functionality. Clean the ```openai_resources``` list to simulate the OpenAI behaviour with the mocking service.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) 

In [10]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"rg-lab-{deployment_name}-001" # change the name to match your naming style
resource_group_location = "westeurope"
apim_resource_name = "apim001"
apim_resource_location = "westeurope"
apim_resource_sku = "Consumption" # "Basicv2"
openai_resources = [ {"name": "openai1", "location": "swedencentral"}, {"name": "openai2", "location": "francecentral"}  ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_lb_config = [ {"name": "openai1", "priority": 1, "weight": 100}, {"name": "openai2", "priority": 2, "weight": 300}  ] # advanced load balancer configuration for OpenAI resources that will be stored as a named value
openai_resources_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ {"name": "openaimock1", "endpoint": "https://openaimock1.azurewebsites.net"}, {"name": "openaimock2", "endpoint": "https://openaimock2.azurewebsites.net"} ]
mock_lb_config = [ {"name": "openaimock1", "priority": 1, "weight": 100}, {"name": "openaimock2", "priority": 2, "weight": 300}  ] # advanced load balancer configuration for mock services that will be stored as a named value


<a id='1'></a>
### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [11]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

✅ Azure Resource Group  rg-lab-advanced-load-balancing-001  created ⌚  13:15:52.834480


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [12]:

loadbalancer_config = openai_lb_config if len(openai_resources) > 0 else mock_lb_config
loadbalancer_config = json.dumps(loadbalancer_config).replace("\"", "'")

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAILoadBalancingConfigValue": { "value": loadbalancer_config },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku}
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

{
  "id": "/subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/rg-lab-advanced-load-balancing-001/providers/Microsoft.Resources/deployments/advanced-load-balancing",
  "location": null,
  "name": "advanced-load-balancing",
  "properties": {
    "correlationId": "c51ab434-7bbb-4560-9973-726a14d2f24f",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
            "id": "/subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/rg-lab-advanced-load-balancing-001/providers/Microsoft.ApiManagement/service/apim001-i7xyvwzuxcng6",
            "resourceGroup": "rg-lab-advanced-load-balancing-001",
            "resourceName": "apim001-i7xyvwzuxcng6",
            "resourceType": "Microsoft.ApiManagement/service"
          }
        ],
        "id": "/subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/rg-lab-advanced-load-balancing-001/providers/Microsoft.ApiManagement/service/apim001-i7xyvwzuxcng6/apis/openai",
     


d:\projects\AI-Gateway\labs\advanced-load-balancing\main.bicep(271,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagnostics#BCP035]




<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [13]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

👉🏻 API Gateway URL:  https://apim001-i7xyvwzuxcng6.azure-api.net


<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses.

In [14]:
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version
if len(openai_resources) > 0:
    messages={"messages":[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}
elif len(mock_webapps) > 0:
    messages={
        "messages": [
            {
                "role": "system", 
                "content": {
                    "simulation": {
                        "default": {"response_status_code": 200, "wait_time_ms": 0},
                        "openaimock1.azurewebsites.net": {"response_status_code": 429}
                    }
                }
            }
        ]
    }
response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)
print("status code: ", response.status_code)
print("headers ", response.headers)
print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
print("x-ms-openai: ", response.headers.get("x-ms-openai")) 

if (response.status_code == 200):
    data = json.loads(response.text)
    print("response: ", data.get("choices")[0].get("message").get("content"))
else:
    print(response.text)

status code:  200
headers  {'Content-Type': 'application/json', 'Date': 'Mon, 16 Dec 2024 12:21:03 GMT', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'no-cache, must-revalidate', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'apim-request-id': '3bdc0f95-14ed-47a7-96a1-4557539420a9', 'X-Content-Type-Options': 'nosniff', 'x-ms-region': 'Sweden Central', 'x-ratelimit-remaining-requests': '19', 'x-ratelimit-remaining-tokens': '19984', 'x-accel-buffering': 'no', 'x-ms-rai-invoked': 'true', 'X-Request-ID': 'b56ba94a-f311-4437-8ceb-9091ba4770d0', 'x-ms-client-request-id': 'Not-Set', 'azureml-model-session': 'd1288-20240926111929', 'x-ms-openai': 'openai1'}
x-ms-region:  Sweden Central
x-ms-openai:  openai1
response:  Oh, I'm more than happy to tell you the time, just as soon as you explain why you can't just look at a clock yourself.


<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK
OpenAPI provides a widely used [Python library](https://github.com/openai/openai-python). The library includes type definitions for all request params and response fields. The goal of this test is to assert that APIM can seamlessly proxy requests to OpenAI without disrupting its functionality.
- Note: run ```pip install openai``` in a terminal before executing this step.

In [15]:
from openai import AzureOpenAI
if len(openai_resources) > 0:
    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]
elif len(mock_webapps) > 0:
    messages=[
            {
                "role": "system", 
                "content": {
                    "simulation": {
                        "default": {"response_status_code": 200, "wait_time_ms": 0},
                        "openaimock1.azurewebsites.net": {"response_status_code": 429}
                    }
                }
            }
        ]
client = AzureOpenAI(
    azure_endpoint=apim_resource_gateway_url,
    api_key=apim_subscription_key,
    api_version=openai_api_version
)
response = client.chat.completions.create(model=openai_model_name, messages=messages)
print(response.choices[0].message.content)

Oh, I would be delighted to tell you the time! But where's the fun in that? Why not use your phone, or a clock, or perhaps the sun as a helpful and reliable timekeeping reference? It's much more entertaining that way, wouldn't you agree?


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.